In [29]:
import pandas as pd
import web3
import time

In [30]:
rpc_url = "https://mainnet.infura.io/v3/c4120217bf7f4dd18189de1e986ff8c1"
provider = web3.providers.rpc.HTTPProvider(rpc_url)
w3 = web3.Web3(provider)

In [31]:
df = pd.read_csv("data.csv")
all_wallet_addresses = set()
cols = df.columns

holders = {}

for col in cols:
    holders[col] = set(df[col].dropna().tolist())

for col in cols:
    for wallet_address in df[col].dropna():
        if all(wallet_address in holders[key] for key in holders.keys()):
            wallet_address = w3.to_checksum_address(wallet_address)
            all_wallet_addresses.add(wallet_address)

In [32]:
len(all_wallet_addresses)

6

In [33]:
all_wallet_addresses

{'0x607538488D455F05F046378c20F078631b184908',
 '0x67011c98e6452332ad3587535aB8c99F9eA78217',
 '0x887a2CA663c35fD607a1Ce6db77e8a5365FE00c4',
 '0x8af712d54B9121B153d4aD3679652FDacD377B4d',
 '0xb8Ff2B58da9cC02747Bf7559fC2a3078F71255B6',
 '0xc7239723911f718277a94D8edd48bb17364809aA'}

In [34]:
# abi that exposes IERC20.balanceOf method
abi = [
    {
        "inputs": [{"internalType": "address", "name": "account", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function",
    }
]

token_addresses = [
    ("OgAndy", "0x748509433ef209c4d11ada51347d5724a5da0ca5"),
    ("SLERF", "0x7f1fe3623c60195233782ee390f98be7e4ed2b47"),
    ("BRANDY", "0xf00d980b525e7cfe351259a37efb72f077396eca"),
]

balances = pd.DataFrame(columns=["Address", "OgAndy", "SLERF", "BRANDY"])
balances.set_index("Address", inplace=True)

num_addresses = 50
# total requests is num_addresses * len(token_addresses)

for token, token_address in token_addresses:
    token_address = w3.to_checksum_address(token_address)
    contract = w3.eth.contract(address=token_address, abi=abi)
    for wallet_address in list(all_wallet_addresses):
        time.sleep(0.1)  # quota for infura is 10 reqs per sec at free plan
        balance = contract.functions.balanceOf(wallet_address).call()
        balances.loc[wallet_address, token] = balance
        print(".", end="")

..................

In [35]:
balances.head()

,OgAndy,SLERF,BRANDY
Address,,,
0x8af712d54B9121B153d4aD3679652FDacD377B4d,2544505515332,5051425078851011825195685,4500000000000000
0xc7239723911f718277a94D8edd48bb17364809aA,10000000000000,7257305689698821416642580,11206532881870034
0x607538488D455F05F046378c20F078631b184908,1650646188834,2426220614336677410654951,4489066019786940
0x887a2CA663c35fD607a1Ce6db77e8a5365FE00c4,15564809019747,8264944705112867560456689,11765098779263107
0x67011c98e6452332ad3587535aB8c99F9eA78217,322115387022,2171105445633882439241728,5183803596238396


In [36]:
balances.to_csv("balances.csv")